In [3]:
from requests_html import HTMLSession
import csv
import pandas as pd

session = HTMLSession()
r = session.get('https://www.cwitkowski.de')

In [4]:
r.html.links

{'../about/',
 '../blog/',
 '../contact/',
 '../cv/',
 '../impressum/',
 '/',
 'https://github.com/gaalcaras/academic',
 'https://jekyllrb.com/',
 'https://www.linkedin.com/in/claudia-witkowski-11435569/'}

In [3]:
page = [''] 
urls = []

for n in [i for i in range(1, 901)]:
    page.append('?page=' + str(n))

for n in page:
    urls.append('https://www.mut-gegen-rechte-gewalt.de/service/chronik-vorfaelle' + n)

In [4]:
data = []

for n in urls:
    
    r = session.get(n)
    
    for element in r.html.find('.node-chronik-eintrag'):

        if element.find('.field-name-field-date', first = True) != None:
            datum = element.find('.field-name-field-date', first = True).text
        else:
            datum = None    

        if element.find('.field-name-field-art', first = True) != None:
            art = element.find('.field-name-field-art', first = True).text
        else:
            art = None    

        if element.find('.field-name-field-anzahl-verletze', first = True) != None:
            verletzte = element.find('.field-name-field-anzahl-verletze', first = True).text
        else:
            verletzte = None    

        if element.find('.field-name-field-city', first = True) != None:
            stadt = element.find('.field-name-field-city', first = True).text
        else:
            stadt = None    

        if element.find('.field-name-field-bundesland', first = True) != None:
            bundesland = element.find('.field-name-field-bundesland', first = True).text
        else:
            bundesland = None        
        
        if element.find('a[href^="http"]', first = True) != None:
            quelle = element.find('a[href^="http"]', first = True).text
        else:
            quelle = None 
        
        if element.find('a[href^="http"]', first = True) != None:
            quelle_url = element.find('a[href^="http"]', first = True).links
        else:
            quelle_url = None 
        
        if element.find('.group-right', first = True) != None:
            beschreibung = element.find('.group-right', first = True).text
        else:
            beschreibung = None  
            
        data.append({'datum':datum, 'art':art, 'verletzte':verletzte, 'stadt':stadt, 'bundesland':bundesland,
                     'quelle':quelle, 'quelle_url':quelle_url, 'beschreibung':beschreibung})

In [5]:
fieldnames = ['datum', 'verletzte', 'art', 'bundesland', 'stadt', 'quelle', 'quelle_url', 'beschreibung']
out_file = open('mut_gegen_rechte_gewalt.csv','w')
csvwriter = csv.DictWriter(out_file, delimiter=',', fieldnames=fieldnames)
csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
for row in data:
     csvwriter.writerow(row)
row_count = sum(1 for row in data)        
print(row_count)
out_file.close()

9006


In [6]:
df = pd.DataFrame(data)

In [7]:
aggr = df.groupby(['bundesland']).size().reset_index(name='counts')

In [8]:
pop = pd.read_csv('query.csv')
pop = pop.drop([10], axis=0) # delete duplicate for Rheinland-Pfalz (keep latest value)
pop.iloc[4,0] = 'Bremen' # rename Bremen
pop = pop.rename(columns={"landLabel": "bundesland"}) # rename 
pop

,bundesland,population
0,Baden-Württemberg,10703000
1,Bayern,12930751
2,Berlin,3611222
3,Brandenburg,2449193
4,Bremen,661000
5,Hamburg,1810438
6,Hessen,6045425
7,Mecklenburg-Vorpommern,1596500
8,Niedersachsen,7945685
9,Nordrhein-Westfalen,17572000


In [9]:
full = pd.merge(aggr, pop)
full

,bundesland,counts,population
0,Baden-Württemberg,805,10703000
1,Bayern,1062,12930751
2,Berlin,698,3611222
3,Brandenburg,928,2449193
4,Bremen,24,661000
5,Hamburg,122,1810438
6,Hessen,252,6045425
7,Mecklenburg-Vorpommern,387,1596500
8,Niedersachsen,644,7945685
9,Nordrhein-Westfalen,967,17572000


In [13]:
# compute counts 
full["proportion"] = full["population"] / full["counts"]
full[["bundesland", "proportion"]].sort_values(by="proportion")

,bundesland,proportion
3,Brandenburg,2639.216595
12,Sachsen,2902.844667
7,Mecklenburg-Vorpommern,4125.322997
13,Sachsen-Anhalt,4150.591497
15,Thüringen,4410.204082
2,Berlin,5173.670487
14,Schleswig-Holstein,7279.146096
1,Bayern,12175.848399
8,Niedersachsen,12338.020186
0,Baden-Württemberg,13295.652174


In [ ]:
full[["bundesland", "proportion"]].sort_values(by="proportion").to_csv('proportion.csv', index = False)